<a href="https://colab.research.google.com/github/YanickSchraner/rl-on-trains-workshop/blob/main/01_Intro_to_environment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Flatland
Docs at: http://flatland-rl-docs.s3-website.eu-central-1.amazonaws.com/


## File structure


```
├── Notebooks, Readme, packages ..
├── agents: RL agents implementation
│   ├── curiosity.py
│   ├── dqn.py
│   ├── qlearning.py
│   └── random.py
├── helpers: Helpers to train, test, inspect agents
│   └── rl_helpers.py
└── videos: Save videos of your best agents here!
    └── video.mp4
```



## Imports
First we need to import all libraries and clone external helper code

In [1]:
#@title << Setup Google Colab by running this cell {display-mode: "form"}
import sys
if 'google.colab' in sys.modules:
    # Clone GitHub repository
    !git clone --single-branch --branch evaluation_setup https://github.com/YanickSchraner/rl-on-trains-workshop
        
    # Copy files required to run the code
    !cp -r "rl-workshop/agents" "rl-workshop/rl_helpers" .
    
    # Install packages via pip
    !pip install -r "rl-workshop/requirements.txt"
    
    # Restart Runtime
    import os
    os.kill(os.getpid(), 9)

In [2]:
%run utils/fast_tree_obs.py

# Import libraries
import ipycanvas
from IPython import display

import time

import numpy as np

# In Flatland you can use custom observation builders and predicitors
# Observation builders generate the observation needed by the controller
# Preditctors can be used to do short time prediction which can help in avoiding conflicts in the network

# First of all we import the Flatland rail environment
from flatland.envs.rail_env import RailEnv
from flatland.envs.rail_generators import sparse_rail_generator
from flatland.envs.observations import TreeObsForRailEnv, GlobalObsForRailEnv
from flatland.envs.schedule_generators import sparse_schedule_generator
from flatland.envs.malfunction_generators import malfunction_from_params, MalfunctionParameters, ParamMalfunctionGen

# We also include a renderer because we want to visualize what is going on in the environment
from flatland.utils.rendertools import RenderTool

In [3]:
%%capture
%matplotlib inline

# Helper function to visualize an episode

import matplotlib.pyplot as plt
import matplotlib.animation
import matplotlib.image as mpimg
plt.rcParams["animation.html"] = "jshtml"

def display_episode(frames):
    fig, ax = plt.subplots(figsize=(12,12))
    imgplot = plt.imshow(frames[0])
    def animate(i):
        imgplot.set_data(frames[i])
    animation = matplotlib.animation.FuncAnimation(fig, animate, frames=len(frames))
    return animation

## Presentation of the RL environment

In the next cells we specify the characteristics of our flatland grid and the trains.
We use the `FastTreeObs` observation for all of our agents.

Training on simple small tasks is the best way to get familiar with the environment.
We started by importing the necessary rail and schedule generators.
The rail generator will generate the railway infrastructure.
The schedule generator will assign tasks to all the agent within the railway network.

The railway infrastructure can be build using any of the provided generators in env/rail_generators.py
Here we use the sparse_rail_generator with the following parameters:



In [4]:
n_agents = 5 # Number of trains that have an assigned task in the env
# Characteristics of the flatland grid:
x_dim = 300 # With of map
y_dim = 300 # Height of map
n_cities = 10 # Number of cities where agents can start or end
max_rails_between_cities = 2 # Max number of tracks allowed between cities. This is number of entry point to a city
max_rails_in_city = 8 # Max number of parallel tracks within a city, representing a realistic trainstation

seed = 42 # Random seed

# Custom observation builder
tree_observation = FastTreeObs(max_depth=2)

The schedule generator can make very basic schedules with a start point, end point and a speed profile for each agent.
The speed profiles can be adjusted directly as well as shown later on. We start by introducing a statistical distribution of speed profiles

In [5]:
# Different agent types (trains) with different speeds.
speed_ration_map = {1.: 0.25,  # Fast passenger train
                    1. / 2.: 0.25,  # Fast freight train
                    1. / 3.: 0.25,  # Slow commuter train
                    1. / 4.: 0.25}  # Slow freight train

# We can now initiate the schedule generator with the given speed profiles
schedule_generator = sparse_schedule_generator(speed_ration_map)

In [6]:
# We can furthermore pass stochastic data to the RailEnv constructor which will allow for stochastic malfunctions
# during an episode.

stochastic_data = MalfunctionParameters(malfunction_rate=1/10000,  # Rate of malfunction occurence
                                        min_duration=15,  # Minimal duration of malfunction
                                        max_duration=50  # Max duration of malfunction
                                        )



Construct the enviornment with the given observation, generataors, predictors, and stochastic data
After creating the environment, we need to call `reset()` to initialize it.

In [7]:
env = RailEnv(width=x_dim,
              height=y_dim,
              rail_generator=sparse_rail_generator(max_num_cities=n_cities,# Number of cities in map (where train stations are)
                                                   seed=seed,  # Random seed
                                                   grid_mode=False,
                                                   max_rails_between_cities=max_rails_between_cities,
                                                   max_rails_in_city=max_rails_in_city,
                                                   ),
              schedule_generator=schedule_generator,
              number_of_agents=n_agents,
              malfunction_generator=ParamMalfunctionGen(stochastic_data),
              obs_builder_object=tree_observation,
              remove_agents_at_target=True,
              record_steps=True
              )

obs = env.reset()

The `RenderTool` allows us to visualize the environment.

In [8]:
env_renderer = RenderTool(env, gl="PGL", screen_width=500, screen_height=500, show_debug=False)

We first look at the map we have created

In [9]:
env_renderer.render_env(show=True)

open_window - pyglet


As an initial example we create a random agent

In [10]:
# Now we can take 500 random steps in our environment and then visualize how the train moved in this episode.
class RandomAgent:

    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size

    def act(self, state):
        """
        :param state: input is the observation of the agent
        :return: returns an action
        """
        return np.random.choice([RailEnvActions.MOVE_FORWARD, RailEnvActions.MOVE_RIGHT, RailEnvActions.MOVE_LEFT,
                                 RailEnvActions.STOP_MOVING])

    def step(self, memories):
        """
        Step function to improve agent by adjusting policy given the observations

        :param memories: SARS Tuple to be
        :return:
        """
        return

    def save(self, filename):
        # Store the current policy
        return

    def load(self, filename):
        # Load a policy
        return


Initialize the agent with the parameters corresponding to the environment and observation_builder

In [11]:
controller = RandomAgent(218, env.action_space[0])

We start by looking at the information of each agent.
We can see the task assigned to the agent by looking at:

In [12]:
print("\n Agents in the environment have to solve the following tasks: \n")
for agent_idx, agent in enumerate(env.agents):
    print(
        "The agent with index {} has the task to go from its initial position {}, facing in the direction {} to its target at {}.".format(
            agent_idx, agent.initial_position, agent.direction, agent.target))


 Agents in the environment have to solve the following tasks: 

The agent with index 0 has the task to go from its initial position (222, 270), facing in the direction 2 to its target at (204, 210).
The agent with index 1 has the task to go from its initial position (222, 268), facing in the direction 0 to its target at (51, 84).
The agent with index 2 has the task to go from its initial position (51, 83), facing in the direction 0 to its target at (218, 192).
The agent with index 3 has the task to go from its initial position (181, 149), facing in the direction 1 to its target at (83, 252).
The agent with index 4 has the task to go from its initial position (153, 253), facing in the direction 0 to its target at (88, 213).


The agent will always have a status indicating if it is currently present in the environment or done or active.
For example we see that agent with index 0 is currently not active

In [13]:
print("\n Their current statuses are:")
print("============================")

for agent_idx, agent in enumerate(env.agents):
    print("Agent {} status is: {} with its current position being {}".format(agent_idx, str(agent.status),
                                                                             str(agent.position)))


 Their current statuses are:
Agent 0 status is: RailAgentStatus.READY_TO_DEPART with its current position being None
Agent 1 status is: RailAgentStatus.READY_TO_DEPART with its current position being None
Agent 2 status is: RailAgentStatus.READY_TO_DEPART with its current position being None
Agent 3 status is: RailAgentStatus.READY_TO_DEPART with its current position being None
Agent 4 status is: RailAgentStatus.READY_TO_DEPART with its current position being None


The agent needs to take any action [1,2,3] except do_nothing or stop to enter the level.

If the starting cell is free they will enter the level.
If multiple agents want to enter the same cell at the same time the lower index agent will enter first.

Let's check if there are any agents with the same start location:

In [14]:
agents_with_same_start = set()
print("\n The following agents have the same initial position:")
print("=====================================================")
for agent_idx, agent in enumerate(env.agents):
    for agent_2_idx, agent2 in enumerate(env.agents):
        if agent_idx != agent_2_idx and agent.initial_position == agent2.initial_position:
            print("Agent {} as the same initial position as agent {}".format(agent_idx, agent_2_idx))
            agents_with_same_start.add(agent_idx)



 The following agents have the same initial position:


Let's try to enter with all of these agents at the same time

In [15]:
action_dict = dict()

for agent_id in agents_with_same_start:
    action_dict[agent_id] = 1  # Try to move with the agents

# Do a step in the environment to see what agents entered:
env.step(action_dict)

({0: array([0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0.]),
  1: array([0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0.]),
  2: array([0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0.]),
  3: array([0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0.]),
  4: array([0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0.])},
 {0: -0.3333333333333333,
  1: -0.3333333333333333,
  2: -0.3333333333333333,
  3: -0.5,
  4: -0.25},
 {0: False, 1: False, 2: False, 3: False, 4: False, '__all__': False},
 {'action_required': {0: True, 1: True, 2: True, 3: True, 4: True},
  'malfunction': {0: 0, 1: 0, 2: 0, 3: 0, 4: 0},
  'speed': {0: 0.3333333333333333,
   1: 0.3333333333333333,
   2: 0.33333

Current state and position of the agents after all agents with same start position tried to move

In [16]:
print("\n This happened when all tried to enter at the same time:")
print("========================================================")
for agent_id in agents_with_same_start:
    print(
        "Agent {} status is: {} with the current position being {}.".format(
            agent_id, str(env.agents[agent_id].status),
            str(env.agents[agent_id].position)))



 This happened when all tried to enter at the same time:


As you see only the agents with lower indexes moved. 
As soon as the cell is free again the agents can attempt to start again.

You will also notice, that the agents move at different speeds once they are on the rail.
The agents will always move at full speed when moving, never a speed inbetween.
The fastest an agent can go is 1, meaning that it moves to the next cell at every time step.

All slower speeds indicate the fraction of a cell that is moved at each time step.
Lets look at the current speed data of the agents:

In [17]:
print("\n The speed information of the agents are:")
print("=========================================")

for agent_idx, agent in enumerate(env.agents):
    print(
        "Agent {} speed is: {:.2f} with the current fractional position being {}".format(
            agent_idx, agent.speed_data['speed'], agent.speed_data['position_fraction']))



 The speed information of the agents are:
Agent 0 speed is: 0.33 with the current fractional position being 0.0
Agent 1 speed is: 0.33 with the current fractional position being 0.0
Agent 2 speed is: 0.33 with the current fractional position being 0.0
Agent 3 speed is: 0.50 with the current fractional position being 0.0
Agent 4 speed is: 0.25 with the current fractional position being 0.0


New the agents can also have stochastic malfunctions happening which will lead to them being unable to move for a certain amount of time steps.
The malfunction data of the agents can easily be accessed as follows

In [18]:
print("\n The malfunction data of the agents are:")
print("========================================")

for agent_idx, agent in enumerate(env.agents):
    print(
        "Agent {} is OK = {}".format(
            agent_idx, agent.malfunction_data['malfunction'] < 1))



 The malfunction data of the agents are:
Agent 0 is OK = True
Agent 1 is OK = True
Agent 2 is OK = True
Agent 3 is OK = True
Agent 4 is OK = True


Now that you have seen these novel concepts that were introduced you will realize that agents don't need to take
an action at every time step as it will only change the outcome when actions are chosen at cell entry.

Therefore the environment provides information about what agents need to provide an action in the next step.

You can access this in the following way:

In [19]:
# Chose an action for each agent
for a in range(env.get_num_agents()):
    action = controller.act(0)
    action_dict.update({a: action})
# Do the environment step
observations, rewards, dones, information = env.step(action_dict)
print("\n The following agents can register an action:")
print("========================================")
for info in information['action_required']:
    print("Agent {} needs to submit an action.".format(info))


NameError: name 'RailEnvActions' is not defined

Let us now look at an episode playing out with random actions performed

In [ ]:
print("\nStart episode...")

# Reset the rendering system
env_renderer.reset()

score = 0
# Run episode
frame_step = 0
frames = []

In [ ]:
for step in range(500):
    # Chose an action for each agent in the environment
    for a in range(env.get_num_agents()):
        action = controller.act(observations[a])
        action_dict.update({a: action})

    # Environment step which returns the observations for all agents, their corresponding
    # reward and whether their are done

    next_obs, all_rewards, done, _ = env.step(action_dict)

    frame = env_renderer.render_env(show=True, show_observations=False, show_predictions=False, return_image=True)
    frames.append(frame)
    frame_step += 1
    # Update replay buffer and train agent
    for a in range(env.get_num_agents()):
        controller.step((observations[a], action_dict[a], all_rewards[a], next_obs[a], done[a]))
        score += all_rewards[a]

    observations = next_obs.copy()
    if done['__all__']:
        break
    print('Episode: Steps {}\t Score = {}'.format(step, score))


Our helper function `display_episode` allows us to get an animated episode. By putting `%%capture` at the begining of the cell we can surpress the cell output and create a clean output in the next cell.

In [ ]:
%%capture
animation = display_episode(frames)

In [ ]:
animation